### ARX MIMO Example
Case: 3 outputs x 4 inputs

Author: Giuseppe Armenise, revised by RBdC

In [1]:
import numpy as np
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline

from sippy_unipi.datasets import load_sample_mimo
from sippy_unipi.io import ARX, FIR
from sippy_unipi.plot import plot_comparison
from sippy_unipi.preprocessing import StandardScaler

# Enable automatic reloading of modules when they change
%load_ext autoreload
%autoreload 2

In [2]:
seed = 0
np.random.seed(0)

In [3]:
na = [3, 1, 2]
nb = [[2, 1, 3, 2], [3, 2, 1, 1], [1, 2, 1, 2]]
th = [[1, 2, 2, 1], [1, 2, 0, 0], [0, 1, 0, 2]]

n_samples = 401
ts = 1.0
time, Ysim, Usim, g_sys, Yerr, Uerr, h_sys, Ytot, Utot = load_sample_mimo(
    n_samples, ts, seed=seed
)

### Identification Parameters

In [4]:
models = [
    Pipeline(
        [
            ("scaler", StandardScaler(with_mean=False)),
            (
                "model",
                TransformedTargetRegressor(
                    ARX(na, nb, th, method="lls"),
                    transformer=StandardScaler(with_mean=False),
                ),
            ),
        ]
    ),
    Pipeline(
        [
            ("scaler", StandardScaler(with_mean=False)),
            (
                "model",
                TransformedTargetRegressor(
                    FIR(nb, th, method="lls"),
                    transformer=StandardScaler(with_mean=False),
                ),
            ),
        ]
    ),
]

Ys_pred = [Ytot]
for model in models:
    model.fit(Utot, Ytot)
    Ys_pred.append(model.predict(Utot))

### Plot Results

In [ ]:
fig = plot_comparison(
    time,
    Usim,
    [r"$u_\mathrm{GBN, " + str(i) + "}$" for i in range(Usim.shape[1])],
    title="Input (Switch probability=0.03) (validation data)",
)

fig = plot_comparison(
    time,
    Ys_pred,
    ylabels=[f"$y_{i}$" for i in range(3)],
    legend=["System", "ARX", "FIR"],
    title="Output (validation data)",
)